In [1]:
from sentence_transformers import SentenceTransformer
from genshin_data import GenshinData
import nltk
import torch
import faiss
import numpy as np

C:\Users\User\AppData\Local\Programs\Python\Python311\Lib\site-packages\sentence_transformers\cross_encoder\CrossEncoder.py:11: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange


In [2]:
genshin_data = GenshinData("characters_data.json", "lore_data.json")
chunks = genshin_data.get_all_chuncks()
len(chunks)

6641

In [3]:
class FaissVectorDB:
    def __init__(self, dimension):
        self.index = faiss.IndexFlatL2(dimension)
        self.documents = []

    def add_documents(self, texts, embeddings):
        embeddings = np.array(embeddings).astype("float32")
        self.index.add(embeddings)
        self.documents.extend(texts)
        print("Added documents to the FAISS index")

    def search(self, query_embedding, top_k=5):
        query_embedding = np.array([query_embedding]).astype("float32")
        distances, indices = self.index.search(query_embedding, top_k)
        return [self.documents[idx] for idx in indices[0]], distances[0]

In [5]:
model = SentenceTransformer('all-MiniLM-L6-v2')

C:\Users\User\AppData\Local\Programs\Python\Python311\Lib\site-packages\transformers\tokenization_utils_base.py:1617: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be deprecated in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [6]:
dimension = 384

faiss_db = FaissVectorDB(dimension)

embeddings = [model.encode(text).tolist() for text in chunks]
faiss_db.add_documents(chunks, embeddings)

C:\Users\User\AppData\Local\Programs\Python\Python311\Lib\site-packages\transformers\models\bert\modeling_bert.py:440: UserWarning: 1Torch was not compiled with flash attention. (Triggered internally at ..\aten\src\ATen\native\transformers\cuda\sdp_utils.cpp:263.)
  attn_output = torch.nn.functional.scaled_dot_product_attention(


Added documents to the FAISS index


In [13]:
query = "Give me story of Li Yue"
query_embedding = model.encode(query)
results, distances = faiss_db.search(query_embedding, top_k=10)

In [14]:
for i, (result, distance) in enumerate(zip(results, distances)):
    print(f"Rank {i+1}\n{result}\nDistance: {distance}\n")

Rank 1
Looking to keep costs down, they lured Xiangling onboard after meeting her on the streets, by hook and by crook. When she saw Beidou, Xiangling was dismayed, and let Beidou know this in no uncertain terms: "If you want to have Yue-style seafood, then I can't agree to help you. I'll have you know that I intend to transcend Liyue's two warring culinary styles with my cooking..."
Distance: 0.8314890265464783

Rank 2
Liyue is home to two rival culinary traditions, conveniently named the "Li" and "Yue" styles. Foodies in Liyue are staunch supporters of one style or the other, and have long been embroiled in a bitter battle to prove the superiority of their style.
Distance: 0.8898718953132629

Rank 3
To her, her father only secured his footing in the forever-ongoing war between the Li and Yue cuisines by upholding that principle.
Distance: 0.9003896713256836

Rank 4
Ever since that incident, Chongyun has found that his reputation precedes him for miles around. He has become beloved in